# 自动化下载10-K或10-Q报告

## 1 Auto download
如果你需要批量下载多家公司的10-K或10-Q报告，或者需要定期获取数据，使用Python等编程语言自动化下载是更高效的方式。以下是具体步骤和示例代码。

pip install requests beautifulsoup4 sec-edgar-downloader pandas

2.2 获取公司CIK
每家上市公司在EDGAR数据库中都有一个唯一的CIK号，用于标识公司。你需要知道目标公司的CIK，可以通过以下方式获取：
在EDGAR主页使用“CIK Lookup”工具，手动搜索公司名称或股票代码。
或者，使用编程方式从SEC提供的CIK列表文件中获取（文件地址：https://www.sec.gov/Archives/edgar/cik-lookup-data.txt）。

2.3 使用sec-edgar-downloader库下载
sec-edgar-downloader是一个专门为EDGAR数据库设计的Python库，可以方便地下载10-K或10-Q报告。以下是示例代码：

In [4]:
from sec_edgar_downloader import Downloader

# 初始化下载器，需提供公司名称和邮箱地址（用于生成符合SEC政策的User-Agent）
# 文件将下载到指定路径（此处为当前目录）
output_folder = "/Users/xhc/Documents/2 Doing/[PhD]AI实验/datasets"
dl = Downloader("XDSISU", "caoxianghan@gmail.com", output_folder)

# 下载Apple（股票代码：AAPL）的所有10-K报告
#dl.get("10-K", "AAPL", limit=3)

# 下载Apple的最近5份10-Q报告
#dl.get("10-Q", "XOM", limit=30)

# 下载Apple在特定日期范围内的10-K报告（日期格式为"YYYY-MM-DD"）
dl.get("10-K", "NEE", after="2015-01-01", before="2016-01-31")

# 下载多个公司的10-K报告（使用股票代码或CIK）
#tickers = ["AAPL", "MSFT", "GOOGL"]  # 公司列表
#for ticker in tickers:
#    dl.get("10-K", ticker, limit=3)  # 每家公司下载最近3份10-K

1

代码说明：

Downloader对象需要提供公司名称和邮箱地址，这是SEC的政策要求，用于生成User-Agent。
dl.get方法的参数：
第一个参数是报告类型（如"10-K"或"10-Q"）。
第二个参数是股票代码或CIK（如"AAPL"或"0000320193"）。
limit参数限制下载的报告数量（可选）。
after和before参数指定日期范围（可选）。
下载的文件将保存到指定路径，默认以公司代码和报告类型组织文件夹结构。

In [5]:
import os
import re

# 1. 定义函数：提取 <DOCUMENT> 之后的内容并保存为 .htm 文件
def process_txt_file(txt_path, output_folder, file_name):
    try:
        # 获取文件名（不含扩展名）
        base_name = os.path.splitext(os.path.basename(txt_path))[0]
        # 定义新的文件名，扩展名为 .htm
        new_file_name = f"{base_name}-{file_name}.htm"
        output_path = os.path.join(output_folder, new_file_name)
        
        # 检查是否已经存在 .htm 文件
        if os.path.exists(output_path):
            print(f"Skipping {txt_path}: {new_file_name} already exists in {output_folder}.")
            return
        
        # 读取 txt 文件内容
        with open(txt_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 查找 <DOCUMENT> 标签的位置
        document_start = content.find('<DOCUMENT>')
        if document_start == -1:
            print(f"Warning: <DOCUMENT> tag not found in {txt_path}. Skipping file.")
            return
        
        # 提取 <DOCUMENT> 之后的内容
        cleaned_content = content[document_start:]
        
        # 保存为新的 .htm 文件
        with open(output_path, 'w', encoding='utf-8') as output_file:
            output_file.write(cleaned_content)
        
        #print(f"Processed {txt_path} -> {output_path}")
    
    except Exception as e:
        print(f"Error processing {txt_path}: {e}")

# 2. 定义函数：批量处理文件夹中的 txt 文件
def process_folder(input_folder, output_folder, file_name):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 遍历输入文件夹中的所有文件
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):  # 只处理 .txt 文件
            txt_path = os.path.join(input_folder, filename)
            process_txt_file(txt_path, output_folder, file_name)
        elif filename.endswith('.htm'):  # 如果是 .htm 文件，跳过
            print(f"Skipping {filename}: File is already in .htm format.")
        else:
            print(f"Skipping {filename}: Unsupported file format.")


In [9]:
import os

folder = "/Users/xhc/Documents/2 Doing/[PhD]AI实验/datasets/sec-edgar-filings/NEE/10-Q/"
output_folder = "/Users/xhc/Documents/2 Doing/[PhD]AI实验/datasets/sec-edgar-filings/NEE/10-Q-htm/"
for i in os.listdir(folder):
    if '.DS_Store' in i: continue
    print(i)
    input_folder = os.path.join(folder, i)
    # 运行批量处理
    process_folder(input_folder, output_folder, i)

0000753308-19-000165
0000753308-17-000081
0000753308-18-000159
0000753308-21-000108
0000753308-20-000150
0000753308-23-000045
0000753308-15-000249
0000753308-17-000109
0000753308-19-000207
0000753308-22-000055
0000753308-23-000058
0000753308-21-000148
0000753308-15-000206
0000753308-20-000186
0000753308-23-000033
0000753308-24-000023
0000753308-24-000050
0000753308-24-000057
0000753308-16-000444
0000753308-15-000166
0000753308-21-000167
0000753308-22-000081
0000753308-18-000081
0000753308-22-000033
0000753308-17-000142
0000753308-20-000113
0000753308-16-000372
0000753308-16-000402
0000753308-18-000114
0000753308-19-000118
